In [1]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings

In [3]:
chromaDirectory = "./chroma_db_nccn"
clientBaseUrl = "http://localhost:1234/v1"
clientApiKey = "not-needed"
sentenceTransformerName = "sentence-transformers/all-MiniLM-L6-v2"

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def print_and_format_docs(docs):
    for doc in docs:
        print(f"Document: {doc}")
    return format_docs(docs)

client = ChatOpenAI(base_url=clientBaseUrl, api_key=clientApiKey)
embedding_function=HuggingFaceEmbeddings(model_name=sentenceTransformerName)
vector_db = Chroma(persist_directory=chromaDirectory, embedding_function=embedding_function)
prompt = hub.pull("rlm/rag-prompt")

retriever = vector_db.as_retriever(search_kwargs={"k": 1})

rag_chain = (
    {"context": retriever | print_and_format_docs, "question": RunnablePassthrough()}
    | prompt
    | client
    | StrOutputParser()
)

rag_chain.invoke("What is a good Greek word to use when translating the word 'Warehouse'")